In [92]:
import numpy as np
import pandas as pd
df=pd.read_csv('test_data.csv', delimiter=',')

Поиск приветствий:

In [145]:
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary
    
    
R1 = or_(
    rule(
        and_(
            gram('ADJF'), 

            dictionary({
            'добрый'
 
                })

                dictionary({
                    'утро',
                    'день', 
                    'вечер'
            })
        ),
    
    rule (dictionary({'привет', 'здравствуйте', 'приветствую'}))
)



parser1 = Parser(R1)

In [146]:
df['has_greeting'] = df['text'].apply(lambda x: bool(len(list(parser1.findall(x)))))

In [147]:
df['has_greeting'] = (df['role']=='manager')*df['has_greeting']

In [148]:
df

,dlg_id,line_n,role,text,has_greeting,has_named,manager_name,company_name,has_goodbye
0,0,0,client,Алло,False,False,,,False
1,0,1,manager,Алло здравствуйте,True,False,,,False
2,0,2,client,Добрый день,False,False,,,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,True,ангелина,диджитал бизнес,False
4,0,4,client,Ага,False,False,,,False
...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,,,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,,,False
477,5,140,client,Спасибо спасибо,False,False,,,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False,,,False


Поиск реплик, где менеджер приставился:

In [99]:
from yargy import rule, or_
from yargy.interpretation import fact
from yargy.predicates import type
from yargy.predicates import gram
from yargy.predicates import normalized
from IPython.display import display
from yargy.relations import main
from yargy.relations import gnc_relation


Name = fact(
    'Name',
    ['first']
)

gnc = gnc_relation()

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ).match(gnc)
).interpretation(
    Name
)


BEFORE = ( 
    rule(
        or_(
            normalized('это'),
            normalized('звать'),
            normalized('я')

        )
    )
)



PERSON = or_(
    rule(BEFORE, NAME)
)

parser3 = Parser(PERSON)


# for match in parser3.findall('меня Миша'):
#     print([_.value for _ in match.tokens])
#     #display(match.rule)



In [100]:
df['has_named'] = df['text'].apply(lambda x: bool(len(list(parser3.findall(x)))))

In [101]:
df['has_named'] = (df['role']=='manager')*df['has_named']

In [102]:
df

,dlg_id,line_n,role,text,has_greeting,has_named
0,0,0,client,Алло,False,False
1,0,1,manager,Алло здравствуйте,True,False
2,0,2,client,Добрый день,False,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,True
4,0,4,client,Ага,False,False
...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False
477,5,140,client,Спасибо спасибо,False,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False


Извлечение имени менеджера

In [121]:
df['manager_name'] = df['text'].apply(lambda x: ''.join(list([_.value for _ in match.tokens][1] for match in parser3.findall(x))))

In [122]:
df

,dlg_id,line_n,role,text,has_greeting,has_named,manager_name,company_name
0,0,0,client,Алло,False,False,,
1,0,1,manager,Алло здравствуйте,True,False,,
2,0,2,client,Добрый день,False,False,,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,True,ангелина,диджитал бизнес
4,0,4,client,Ага,False,False,,
...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,,
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,,
477,5,140,client,Спасибо спасибо,False,False,,
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False,,


Извлечение название компании

In [105]:


Company = fact(
    'Company',
    ['first']
)

COMPANIES = morph_pipeline([
    'диджитал бизнес',
    'китобизнес'
])


BEFORE = ( 
    rule(
        or_(
            normalized('компания')
        )
    )
)


gnc = gnc_relation()

COMPANY = rule(
    COMPANIES.interpretation(
        Company.first.inflected()
    ).match(gnc)
).interpretation(
    Company
)



RES = or_ (
        rule ( BEFORE, COMPANY
    )
)




parser4 = Parser(RES)


for match in parser4.findall('компания диджитал бизнес'):
    print([_.value for _ in match.tokens])
    #display(match.rule)


['компания', 'диджитал', 'бизнес']


In [123]:
df['company_name'] = df['text'].apply(lambda x: ''.join(list(' '.join( str(e) for e in [_.value for _ in match.tokens][1:]) for match in parser4.findall(x))))

In [128]:
df[250:300]

,dlg_id,line_n,role,text,has_greeting,has_named,manager_name,company_name
250,3,1,manager,Алло дмитрий добрый день,True,False,,
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,False,True,максим,китобизнес
252,3,3,client,Да удобно,False,False,,
253,3,4,manager,Да дмитрий вот мне моя коллега анастасия подск...,False,False,,
254,3,5,client,А задача в общем то,False,False,,
255,3,6,client,Стоит из чего сейчас секунду,False,False,,
256,3,7,manager,Угу,False,False,,
257,3,8,client,В общем,False,False,,
258,3,9,client,Настроить воронки как бы и все такое я могу сам,False,False,,
259,3,10,client,Вот и даже сделать этого сэйл с ботом,False,False,,


Извлекать реплики, где менеджер попрощался

In [ ]:
R1 = or_(rule(
            and_(
                dictionary({
                    'до'
 
                    })
                ),
                dictionary({
                    'свидания'

                })
        ),
        rule(
            dictionary({
                'пока'
            }),
        
        ),
         rule(
            and_(
                dictionary({
                    'всего'
 
                    })
                ),
                dictionary({
                    'хорошего'

                })
        )
)




parser5 = Parser(R1)

# for match in parser5.findall('до свидания алина'):
#     print([_.value for _ in match.tokens])

In [151]:
df['has_goodbye'] = df['text'].apply(lambda x: bool(len(list(parser5.findall(x)))))

In [152]:
df['has_goodbye'] = (df['role']=='manager')*df['has_goodbye']

In [153]:
df

,dlg_id,line_n,role,text,has_greeting,has_named,manager_name,company_name,has_goodbye
0,0,0,client,Алло,False,False,,,False
1,0,1,manager,Алло здравствуйте,True,False,,,False
2,0,2,client,Добрый день,False,False,,,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,True,ангелина,диджитал бизнес,False
4,0,4,client,Ага,False,False,,,False
...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,,,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,,,False
477,5,140,client,Спасибо спасибо,False,False,,,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False,,,False


Проверка требований к менеджеру

In [168]:
for i in df.groupby(df['dlg_id']):
    if sum(i[1]['has_greeting'])>0 and sum(i[1]['has_goodbye'])>0:
        print("dialog: "+ str(i[0]) +" is OK")
    else:
        print("dialog: "+ str(i[0]) +" is BAD")

dialog: 0 is OK
dialog: 1 is OK
dialog: 2 is OK
dialog: 3 is OK
dialog: 4 is BAD
dialog: 5 is BAD
